In [1]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import fastf1
import pandas as pd

def rotate(xy, *, angle):
    rot_mat = np.array([[np.cos(angle), np.sin(angle)],
                        [-np.sin(angle), np.cos(angle)]])
    return np.matmul(xy, rot_mat)

In [3]:
year=2023
race='Hungarian Grand Prix'

session = fastf1.get_session(year, race, 'Q')
session.load()

lap = session.laps.pick_fastest()
pos = lap.get_pos_data()

circuit_info = session.get_circuit_info()

core           INFO 	Loading data for Hungarian Grand Prix - Qualifying [v3.3.8]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INF

In [4]:
def rotate(xy, *, angle):
    rot_mat = np.array([[np.cos(angle), np.sin(angle)],
                        [-np.sin(angle), np.cos(angle)]])
    return np.matmul(xy, rot_mat)

In [5]:
# Get an array of shape [n, 2] where n is the number of points and the second
# axis is x and y.
track = pos.loc[:, ('X', 'Y')].to_numpy()

# Convert the rotation angle from degrees to radian.
track_angle = circuit_info.rotation / 180 * np.pi

# Rotate and plot the track map.
rotated_track = rotate(track, angle=track_angle)
df=pd.DataFrame(rotated_track)
df.columns=["x","y"]
fig=px.line(df, 
            x="x", 
            y="y",
            template="simple_white",
            title="<b>{} {} Track Layout</b>".format(year,race)
            )

offset_vector = [500, 0]  # offset length is chosen arbitrarily to 'look good'

# Iterate over all corners.
for _, corner in circuit_info.corners.iterrows():
    # Create a string from corner number and letter
    txt = f"{corner['Number']}{corner['Letter']}"

    # Convert the angle from degrees to radian.
    offset_angle = corner['Angle'] / 180 * np.pi

    # Rotate the offset vector so that it points sideways from the track.
    offset_x, offset_y = rotate(offset_vector, angle=offset_angle)

    # Add the offset to the position of the corner
    text_x = corner['X'] + offset_x
    text_y = corner['Y'] + offset_y

    # # Rotate the text position equivalently to the rest of the track map
    text_x, text_y = rotate([text_x, text_y], angle=track_angle)
    x_text=pd.DataFrame(pd.Series(text_x))
    y_text=pd.DataFrame(pd.Series(text_y))
    text_df=pd.concat([x_text,y_text],axis=1)
    text_df.columns=["x","y"]
    text_df["txt"]=txt

    # Rotate the center of the corner equivalently to the rest of the track map
    track_x, track_y = rotate([corner['X'], corner['Y']], angle=track_angle)
    x_track=pd.DataFrame(pd.Series(track_x))
    y_track=pd.DataFrame(pd.Series(track_y))
    track_df=pd.concat([x_track,y_track],axis=1 )
    track_df.columns=["x","y"]

    # Draw a circle next to the track.
    fig2=px.scatter(text_df, x="x", y="y",color_discrete_sequence=['black'], text='txt')
    fig2.update_traces(marker_size=15)

    fig.add_traces(
    list(fig2.select_traces())
    )

    fig.add_shape(type="line", x0=text_df["x"].values[0]+0, y0=text_df["y"].values[0]+0, x1=track_df["x"].values[0], y1=track_df["y"].values[0], line_width=2, line_dash="solid", line_color="grey")
    # fig.add_shape(type="line", x0=186.249866, y0=-2304.237881, x1=216.207531, y1=-2803.339613, line_width=2, line_dash="dash", line_color="grey")

    # Draw a line from the track to the circle.
    # fig3=px.line([track_x, text_x], [track_y, text_y])

# plt.title(session.event['Location'])
# plt.xticks([])
# plt.yticks([])
# plt.axis('equal')
# plt.show()
fig.update_xaxes(title_text='', showticklabels=False,visible=False)      # Remove axis titles
fig.update_yaxes(title_text='', showticklabels=False,visible=False)
fig.update_traces(textposition='middle center',textfont_color='white')


fig.update_layout(
    height=450, width=650, 
    title_x=0.5,
    margin=dict(l=5, r=5, t=35, b=5),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.show()


In [6]:
fig.write_html("{} {} Track Layout.html".format(year,race),full_html=False, include_plotlyjs='cdn')

In [ ]:
#multi gp plot

In [14]:
#2023
race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
year_multi=2023
for i in race_multi:
    session = fastf1.get_session(year_multi, i, 'FP1')
    session.load()

    lap = session.laps.pick_fastest()
    pos = lap.get_pos_data()

    circuit_info = session.get_circuit_info()
    # Get an array of shape [n, 2] where n is the number of points and the second
    # axis is x and y.
    track = pos.loc[:, ('X', 'Y')].to_numpy()

    # Convert the rotation angle from degrees to radian.
    track_angle = circuit_info.rotation / 180 * np.pi

    # Rotate and plot the track map.
    rotated_track = rotate(track, angle=track_angle)
    df=pd.DataFrame(rotated_track)
    df.columns=["x","y"]
    fig=px.line(df, 
                x="x", 
                y="y",
                template="simple_white",
                title="<b>{} {} Track Layout</b>".format(year_multi,i)
                )

    offset_vector = [500, 0]  # offset length is chosen arbitrarily to 'look good'

    # Iterate over all corners.
    for _, corner in circuit_info.corners.iterrows():
        # Create a string from corner number and letter
        txt = f"{corner['Number']}{corner['Letter']}"

        # Convert the angle from degrees to radian.
        offset_angle = corner['Angle'] / 180 * np.pi

        # Rotate the offset vector so that it points sideways from the track.
        offset_x, offset_y = rotate(offset_vector, angle=offset_angle)

        # Add the offset to the position of the corner
        text_x = corner['X'] + offset_x
        text_y = corner['Y'] + offset_y

        # # Rotate the text position equivalently to the rest of the track map
        text_x, text_y = rotate([text_x, text_y], angle=track_angle)
        x_text=pd.DataFrame(pd.Series(text_x))
        y_text=pd.DataFrame(pd.Series(text_y))
        text_df=pd.concat([x_text,y_text],axis=1)
        text_df.columns=["x","y"]
        text_df["txt"]=txt

        # Rotate the center of the corner equivalently to the rest of the track map
        track_x, track_y = rotate([corner['X'], corner['Y']], angle=track_angle)
        x_track=pd.DataFrame(pd.Series(track_x))
        y_track=pd.DataFrame(pd.Series(track_y))
        track_df=pd.concat([x_track,y_track],axis=1 )
        track_df.columns=["x","y"]

        # Draw a circle next to the track.
        fig2=px.scatter(text_df, x="x", y="y",color_discrete_sequence=['black'], text='txt')
        fig2.update_traces(marker_size=15)

        fig.add_traces(
        list(fig2.select_traces())
        )

        fig.add_shape(type="line", x0=text_df["x"].values[0]+0, y0=text_df["y"].values[0]+0, x1=track_df["x"].values[0], y1=track_df["y"].values[0], line_width=2, line_dash="solid", line_color="grey")


    fig.update_xaxes(title_text='', showticklabels=False,visible=False)      # Remove axis titles
    fig.update_yaxes(title_text='', showticklabels=False,visible=False)
    fig.update_traces(textposition='middle center',textfont_color='white')


    fig.update_layout(
        height=450, width=650, 
        title_x=0.5,
        margin=dict(l=5, r=5, t=35, b=5),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig.write_html("{} {} Track Layout.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')

core           INFO 	Loading data for Azerbaijan Grand Prix - Practice 1 [v3.3.8]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            IN

In [88]:
# # Get an array of shape [n, 2] where n is the number of points and the second
# # axis is x and y.
# track = pos.loc[:, ('X', 'Y')].to_numpy()

# # Convert the rotation angle from degrees to radian.
# track_angle = circuit_info.rotation / 180 * np.pi

# # Rotate and plot the track map.
# rotated_track = rotate(track, angle=track_angle)
# plt.plot(rotated_track[:, 0], rotated_track[:, 1])

# offset_vector = [500, 0]  # offset length is chosen arbitrarily to 'look good'

# # Iterate over all corners.
# for _, corner in circuit_info.corners.iterrows():
#     # Create a string from corner number and letter
#     txt = f"{corner['Number']}{corner['Letter']}"

#     # Convert the angle from degrees to radian.
#     offset_angle = corner['Angle'] / 180 * np.pi

#     # Rotate the offset vector so that it points sideways from the track.
#     offset_x, offset_y = rotate(offset_vector, angle=offset_angle)

#     # Add the offset to the position of the corner
#     text_x = corner['X'] + offset_x
#     text_y = corner['Y'] + offset_y

#     # # Rotate the text position equivalently to the rest of the track map
#     text_x, text_y = rotate([text_x, text_y], angle=track_angle)

#     # Rotate the center of the corner equivalently to the rest of the track map
#     track_x, track_y = rotate([corner['X'], corner['Y']], angle=track_angle)

#     # Draw a circle next to the track.
#     plt.scatter(text_x, text_y, color='grey', s=140)

#     # Draw a line from the track to this circle.
#     plt.plot([track_x, text_x], [track_y, text_y], color='grey')

#     # Finally, print the corner number inside the circle.
#     plt.text(text_x, text_y, txt,
#              va='center_baseline', ha='center', size='small', color='white')
# plt.title(session.event['Location'])
# plt.xticks([])
# plt.yticks([])
# plt.axis('equal')
# plt.show()
